In [1]:
'''
step 1. import necessary libraries
'''

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import gensim

from gensim import corpora
import string
from pathlib import Path
from pprint import pprint


D:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
'''
step 2. read in files (from directory) for analysis
'''

#r is the raw sting literals so that windows path slash won't create problem 
data_folder = Path(r'news')

#read each file from the directory into an array and name it corpus
corpus = []
filenames = []

for filename in data_folder.iterdir():
    #encoding for macbook: encoding = "ISO-8859-1"
    fp = open(str(filename), 'r', encoding = "ISO-8859-1")
    corpus.append(fp.read())
    
    #keep the filename for later use
    filenames.append(filename.name)
    fp.close()

print(corpus.__len__())

250


In [3]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Evy
[nltk_data]     Lim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Evy
[nltk_data]     Lim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
'''
step3. conduct preprocessing steps
'''

stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

In [5]:
def clean(doc):
    punc_free = ''.join([ch for ch in doc.lower() if ch not in exclude])
    stop_free = ' '.join([i for i in punc_free.split() if i not in stop]) 
    normalized = ' '.join(lemma.lemmatize(word) for word in stop_free.split())
    return normalized
doc_clean = [clean(doc).split() for doc in corpus]

In [6]:
'''
step 4. prepare word representation - term frequency or doc term matrix
'''

dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [7]:
'''
step 5. create lda model
'''

topic_num = 5
word_num = 5
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics = topic_num, id2word = dictionary, passes=20)
pprint(ldamodel.print_topics(num_topics=topic_num, num_words=word_num))

[(0,
  '0.018*"film" + 0.010*"best" + 0.008*"said" + 0.008*"award" + 0.006*"year"'),
 (1, '0.012*"said" + 0.006*"year" + 0.005*"u" + 0.004*"chelsea" + 0.004*"new"'),
 (2, '0.018*"said" + 0.008*"mr" + 0.007*"would" + 0.004*"year" + 0.003*"new"'),
 (3,
  '0.016*"said" + 0.008*"mr" + 0.007*"would" + 0.005*"people" + 0.004*"also"'),
 (4,
  '0.011*"said" + 0.007*"year" + 0.006*"game" + 0.005*"people" + 0.004*"new"')]


In [8]:
'''
step 6. visualise topics and keywords
'''

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)
vis

D:\Anaconda3\lib\site-packages\matplotlib\colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
D:\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.032600 -0.120335       1        1  21.010405
4     -0.039981 -0.012399       2        1  20.964298
3     -0.040300  0.074539       3        1  20.693897
0      0.161748  0.006786       4        1  19.859890
2     -0.048868  0.051409       5        1  17.471504, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
73    Default  228.000000        film  228.000000  30.0000  30.0000
1048  Default  120.000000        best  120.000000  29.0000  29.0000
3205  Default   83.000000       award   83.000000  28.0000  28.0000
3198  Default   61.000000       actor   61.000000  27.0000  27.0000
3246  Default   56.000000       oscar   56.000000  26.0000  26.0000
272   Default  220.000000          mr  220.000000  25.0000  25.0000
2748  Default   64.000000     chelsea   64.000000  24.0000  24.0000
3528  Default   48.000000        star   48.000000  23.0000  23.0000
3303  Default   47.000000       movie   47.000000  22.0000  22.0000
3199  Default   28.000000     actress   28.000000  21.0000  21.0000
5255  Default   44.000000       blair   44.000000  20.0000  20.0000
353   Default   73.000000    director   73.000000  19.0000  19.0000
2632  Default  106.000000        game  106.000000  18.0000  18.0000
3253  Default   25.000000       prize   25.000000  17.0000  17.0000
3204  Default   29.000000     aviator   29.000000  16.0000  16.0000
152   Default  669.000000        said  669.000000  15.0000  15.0000
1810  Default   29.000000   hollywood   29.000000  14.0000  14.0000
3255  Default   23.000000         ray   23.000000  13.0000  13.0000
3244  Default   23.000000  nomination   23.000000  12.0000  12.0000
421   Default  216.000000       would  216.000000  11.0000  11.0000
2778  Default   26.000000        baby   26.000000  10.0000  10.0000
1637  Default   44.000000       party   44.000000   9.0000   9.0000
8256  Default   33.000000          pc   33.000000   8.0000   8.0000
3830  Default   20.000000       named   20.000000   7.0000   7.0000
91    Default   46.000000      growth   46.000000   6.0000   6.0000
2529  Default   47.000000         cup   47.000000   5.0000   5.0000
5088  Default   41.000000        goal   41.000000   4.0000   4.0000
6536  Default   24.000000        hunt   24.000000   3.0000   3.0000
5696  Default   20.000000        ukip   20.000000   2.0000   2.0000
3264  Default   20.000000       swank   20.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
7129   Topic5   11.623630    ferguson   18.268097   1.2925  -6.6590
798    Topic5   17.162865       brown   32.381485   1.1098  -6.2692
421    Topic5   59.223576       would  216.960403   0.4462  -5.0307
1381   Topic5   16.963200          eu   33.200111   1.0731  -6.2809
370    Topic5   14.765313         law   28.248657   1.0958  -6.4197
5255   Topic5   18.904501       blair   44.862175   0.8804  -6.1726
3530   Topic5   12.405803        test   22.487480   1.1498  -6.5938
550    Topic5   25.882713       world   94.623619   0.4483  -5.8584
1881   Topic5   16.576181      labour   41.778908   0.8202  -6.3040
380    Topic5   16.512362        news   41.716797   0.8178  -6.3079
268    Topic5   19.595354    minister   63.605560   0.5672  -6.1367
404    Topic5   24.514847         say  101.321938   0.3256  -5.9127
415    Topic5   23.054958        told   90.415253   0.3781  -5.9741
275    Topic5   31.088774         new  179.208496  -0.0071  -5.6751
653    Topic5   24.005713          uk  102.788597   0.2902  -5.9337
158    Topic5   20.954363     service   76.267937   0.4527  -6.0696
606    Topic5   19.426800        home   64.356125   0.5468  -6.1453
326    Topic5   32.891903        year  269.566345  -0.3590  -5.6188
2495   Topic5   18.936258         bbc   66.147903   0.4938  -6.1709
1275   Topic5   17.016989        plan   53.